In [ ]:
from analyse_trips import *
import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import folium
from folium import plugins

In [ ]:
gdf = load_data("mapstr_2024_04_24.geojson")
#   https://www.youtube.com/watch?v=xu1wRfUHtKg 
plot_interactive_map(gdf)

In [ ]:
gdf.head()

In [ ]:
# drop column a
gdf.drop(columns="userComment", inplace=True)
gdf.drop(columns="icon", inplace=True)
gdf.head()

In [ ]:
unpack_tag(gdf["tags"])

In [ ]:
plot_interactive_map(load_data(FRENCH_DEPARTEMENTS))

In [ ]:
plot_interactive_map(load_data(WORLD_COUNTRIES))


In [ ]:
plot_interactive_map(load_data("data/german_lander.geojson"))

In [ ]:
world_countries_gdf = load_data(WORLD_COUNTRIES)
world_countries_gdf

In [ ]:
joined = gpd.sjoin(gdf, world_countries_gdf, how="inner", predicate='intersects')
print(len(set(joined["french_short"])))
joined

In [ ]:
gdf["country"] = joined["name_right"]
gdf["country_FR"] = joined["french_short"]
gdf["continent"] = joined["continent"]
gdf


In [ ]:
departements_gdf = load_data(FRENCH_DEPARTEMENTS)
departements_gdf

In [ ]:
joined = gpd.sjoin(gdf, departements_gdf, how="inner", predicate='intersects')
joined

In [ ]:
gdf["departement"] = joined["nom"]
gdf

In [ ]:
unpack_tag(gdf["tags"])

In [ ]:
plot_interactive_map(load_data(FRENCH_DEPARTEMENTS))

In [ ]:
plot_interactive_map(load_data(WORLD_COUNTRIES))


In [ ]:
world_countries_gdf = load_data(WORLD_COUNTRIES)
world_countries_gdf

In [ ]:
joined = gpd.sjoin(gdf, world_countries_gdf, how="inner", predicate='intersects')
print(len(set(joined["french_short"])))
joined

In [ ]:
gdf["country"] = joined["name_right"]
gdf["country_FR"] = joined["french_short"]
gdf["continent"] = joined["continent"]
gdf


In [ ]:
departements_gdf = load_data(FRENCH_DEPARTEMENTS)
departements_gdf

In [ ]:
joined = gpd.sjoin(gdf, departements_gdf, how="inner", predicate='intersects')
joined

In [ ]:
gdf["departement"] = joined["nom"]
gdf

In [ ]:
visited_departements = set(gdf["departement"])
visited_departements.remove(np.nan)
len(visited_departements)

In [ ]:
gdf = gdf.to_crs("EPSG:4087")
world_countries_gdf = world_countries_gdf.to_crs("EPSG:4087")

set(gdf.distance(world_countries_gdf.unary_union))

In [ ]:
world_union = world_countries_gdf.unary_union

nearest_countries = []
for point in gdf["geometry"]:
    nearest_geom = shp.ops.nearest_points(point, world_union)[1]
    nearest_country = world_countries_gdf[
        world_countries_gdf["geometry"] == nearest_geom
    ]#["name"].values[0]
    
    nearest_countries.append(nearest_country)
    
nearest_countries

In [ ]:
from shapely import STRtree

tree = STRtree(world_countries_gdf["geometry"])
nearest_countries = []
for point in gdf["geometry"]:
    # point = gdf["geometry"].iloc[249]
    nearest_geom = tree.nearest(point)
    # break
    nearest_country = world_countries_gdf.loc[nearest_geom]

    nearest_countries.append(nearest_country)
# world_countries_gdf.loc[nearest_geom]
country = gpd.GeoDataFrame(nearest_countries, index=gdf.index)


## Graphical representations with Folium

In [ ]:
gdf = clean_mapstr_data(load_data("mapstr_2024_04_24.geojson"))

map = folium.Map(location=[15, -20], tiles="Cartodb dark_matter", zoom_start=2)


heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf.to_crs(crs=4326).geometry]

heat_data
plugins.HeatMap(heat_data).add_to(map)

map

In [ ]:
gdf_france = gdf[gdf["country"] == "France"]
gdf_france.explore()

In [ ]:
departements_dict = {}
for dep in gdf_france["departement"]:
    if dep is not np.nan:
        try:
            departements_dict[dep] += 1
        except KeyError:
            departements_dict[dep] = 1
# departements_dict = set(gdf_france["departement"])
# departements_dict.remove(np.nan)
gdf_departements = departements_gdf[departements_gdf["nom"].isin(departements_dict.keys())]
gdf_departements["count"] = [departements_dict[dep] for dep in gdf_departements["nom"]]
gdf_departements.explore("count", scheme="natural_breaks", k=9, cmap="magma_r")

In [ ]:
# folium plot of the visited departements
map = folium.Map(location=[47.1, 2.4], zoom_start=6)
# add all departements polygons
for i, row in gdf_departements.iterrows():
    folium.GeoJson(row["geometry"]).add_to(map)
map

In [ ]:
gdf_us = gdf[gdf["country"] == "United States of America"]
us_states_gdf = load_data("data/us_states.geojson")

us_states_dict = {}
for dep in gdf_us["US_state"]:
    if dep is not np.nan:
        try:
            us_states_dict[dep] += 1
        except KeyError:
            us_states_dict[dep] = 1

gdf_us_states_visited = us_states_gdf[us_states_gdf["NAME"].isin(us_states_dict.keys())]
gdf_us_states_visited["count"] = [us_states_dict[dep] for dep in gdf_us_states_visited["NAME"]]
gdf_us_states_visited.explore("count", scheme="natural_breaks", k=9, cmap="magma_r")